Imports

In [1]:
import os
import re
from py2neo import Graph

Variablen

In [2]:
# Global variables 
home_dir = os.getcwd()
folder_path_cooccs = 'Data\\TRC\\cooccs'
folder_path_linkage = 'Data\\TRC\\linkage'
folder_path_output = 'Data\\TRC\\TRCcompare'
abs_folder_path_cooccs = os.path.join(home_dir, folder_path_cooccs)
abs_folder_path_linkage = os.path.join(home_dir, folder_path_linkage)
abs_folder_path_output = os.path.join(home_dir, folder_path_output)

In [3]:
# Connect to the Neo4j database
graph = Graph("bolt://localhost:7687")

Methoden

In [4]:
# Function to compare the strings in two files
def compare_files(file1, file2):
    pattern = r".*?_"
    
    with open(file1, 'r') as f1, open(file2, 'r') as f2:
        # Read the first lines from both files and remove leading/trailing whitespaces
        line1 = f1.readline().strip()
        line2 = f2.readline().strip()

        # Extract the desired strings (String 1 and String 2) from both lines
        str1_f1, str2_f1, str3_f1 = line1.split(';')
        str1_f1 = re.sub(pattern, "", str1_f1)
        str1_f2, str2_f2, str3_f2 = line2.split(';')
        str1_f2 = re.sub(pattern, "", str1_f2)

        # Compare the extracted strings, check for empty strings and return the results
        return str1_f1 == str1_f2, str2_f1 == str2_f2, len(str2_f1) == 0, len(str2_f2) == 0, str1_f1, str2_f1, str2_f2

In [5]:
# Get nodes based on their name attributes
def get_node_by_name(name):
    # Clean the string
    name = name.replace("'", '')
    # Create a Cypher query to find a node with the given name attribute
    cypher_query = f"MATCH (n {{name: '{name}'}}) RETURN n LIMIT 1"
    # Run the Cypher query and retrieve the result as a single node
    result = graph.run(cypher_query).evaluate()
    return result

In [6]:
# Find the shortest path between the nodes
def find_shortest_path(start_name, end_name):
    # Get the start and end nodes using the provided names
    start_node = get_node_by_name(start_name)
    end_node = get_node_by_name(end_name)
    
    #print(start_node, end_node)
    #print(f"Node 1: " + start_name)
    #print(f"Node 2: " + end_name)
    
    # Check if either start or end node is not found in the database
    if start_node is None or end_node is None:
        return None
    
    # Create a Cypher query to find the shortest path between the nodes
    cypher_query = (
        f"MATCH "
        "(TRC1:SINGLE_NODE {name: $start_name}), "
        "(TRC2:SINGLE_NODE {name: $end_name}), "
        "p = shortestPath((TRC1)-[*]-(TRC2)) "
        #"WHERE TRC1<>TRC2 "
        "RETURN length(p) AS length"
    )
    
    # Run the Cypher query and retrieve the shortest path length as the result
    result = graph.run(cypher_query, start_name=start_name, end_name=end_name).evaluate()

    return int(result) if result is not None else None


Main

In [7]:
folder1 = abs_folder_path_cooccs
folder2 = abs_folder_path_linkage


ComparedTerms = 0
SameTRCs = 0
TRCPath = 0
LastPath = 0
NodeNotFoundCount = 0
TRCsNotFound1 = 0
TRCsNotFound2 = 0

# Iterate through the files in both folders simultaneously
for filename1, filename2 in zip(os.listdir(folder1), os.listdir(folder2)):
    file1 = os.path.join(folder1, filename1)
    file2 = os.path.join(folder2, filename2)

    # Compare the strings in the current pair of files
    is_same_str1, is_same_str2, str2_f1_empty, str2_f2_empty, str1_f1, str2_f1, str2_f2 = compare_files(file1, file2)

    # Count missing TRCs
    if str2_f1_empty:
        TRCsNotFound1 += 1
    if str2_f2_empty:
        TRCsNotFound2 += 1

    # print(is_same_str1, is_same_str2, str2_f1, str2_f2, str1_f1)

    # Update counters based on comparison results
    if is_same_str1:
        ComparedTerms += 1

    if is_same_str2:
        SameTRCs += 1

    # Get path length between TRCs
    if not is_same_str2:
        LastPath = "NodeNotFound"
        Path = find_shortest_path(str2_f1, str2_f2)
        #print(Path)
        if Path is not None:
            LastPath = Path
            TRCPath += Path 
        if Path is None:
            NodeNotFoundCount += 1
            print(f"Not found in Graph: {str2_f2}")

    # Write the output file
    output_filename = f'TRCCompare_{str1_f1}.txt'
    with open(os.path.join(abs_folder_path_output, output_filename), 'w') as output_file:
        output_file.write(f"{str2_f1};{'[TRC_identical]' if is_same_str2 else str2_f2};{'Node distance: '+str(LastPath)}\n")

ActualComparisons = ComparedTerms - NodeNotFoundCount
NodeNotInGraph = NodeNotFoundCount - (TRCsNotFound1 + TRCsNotFound2)
PercentTRCSame = SameTRCs/ActualComparisons * 100
AveragePathAll = TRCPath/(ActualComparisons)
if SameTRCs != ActualComparisons:
    AveragePathDiff = TRCPath/(ActualComparisons-SameTRCs)
else:
    AveragePathDiff = 0

# Print the final results
print(f"Compared Terms: {ComparedTerms}")
print(f"Normal TRCs not found by toolbox: {TRCsNotFound1}")
print(f"Linkage TRCs not found by toolbox: {TRCsNotFound2}")
print(f"TRC Nodes not found inside the graph: {NodeNotInGraph}")
print(f"\n")
print(f"TRCs that could be compared: {ActualComparisons}")
print(f"Same TRCs: {SameTRCs}")
print(f"% of TRCs identical: {PercentTRCSame}")
print(f"Average path length between all TRCs: {AveragePathAll}")
print(f"Average path length between differing TRCs: {AveragePathDiff}")

Not found in Graph: Plastics and pollution
Not found in Graph: Districts of Indonesia
Not found in Graph: The Guardian
Not found in Graph: The Guardian
Not found in Graph: Gulf of Mexico
Not found in Graph: Plastics and pollution
Not found in Graph: Environmental effects of shipping
Not found in Graph: Gulf of Mexico
Compared Terms: 78
Normal TRCs not found by toolbox: 0
Linkage TRCs not found by toolbox: 0
TRC Nodes not found inside the graph: 8


TRCs that could be compared: 70
Same TRCs: 0
% of TRCs identical: 0.0
Average path length between all TRCs: 1.5
Average path length between differing TRCs: 1.5
